## Description:
这个Jupyter用Pytorch实现GMF模型， 完成该模型的预训练过程。

## 导入包

In [1]:
import datetime
import numpy as np
import pandas as pd
from collections import Counter
import heapq

import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchkeras import summary

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# 一些超参数设置
topK = 10
num_factors = 8
num_negatives = 4
batch_size = 64
lr = 0.001

## 导入数据

In [3]:
# 数据在processed Data里面
train = np.load('ProcessedData/train.npy', allow_pickle=True).tolist()
testRatings = np.load('ProcessedData/testRatings.npy').tolist()
testNegatives = np.load('ProcessedData/testNegatives.npy').tolist()

In [5]:
num_users, num_items = train.shape
print(num_users)
print(num_items)

6040
3706


In [5]:
# 制作数据   用户打过分的为正样本， 用户没打分的为负样本， 负样本这里采用的采样的方式
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [], [], []
    num_items = train.shape[1]
    for (u, i) in train.keys():  # train.keys()是打分的用户和商品       
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        
        # negative instance
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train:
                j = np.random.randint(num_items)
            #print(u, j)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [6]:
user_input, item_input, labels = get_train_instances(train, num_negatives)

In [7]:
train_x = np.vstack([user_input, item_input]).T
labels = np.array(labels)

In [8]:
# 构建成Dataset和DataLoader
train_dataset = TensorDataset(torch.tensor(train_x), torch.tensor(labels).float())
dl_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [9]:
# 测试一下
for (x, y) in iter(dl_train):
    print(x, y)
    break

tensor([[2051, 3584],
        [1726, 3025],
        [1180, 3073],
        [1171, 1554],
        [2179, 1448],
        [3759,  215],
        [3545, 2908],
        [1883, 2756],
        [3768,   41],
        [1721, 3225],
        [2984, 2192],
        [2906, 1232],
        [1910, 1899],
        [3823,  355],
        [1644, 1351],
        [ 972, 2711],
        [4270, 1031],
        [3791, 2890],
        [3694,  686],
        [4271, 2576],
        [1644, 3058],
        [2450,  102],
        [4807,  737],
        [5141, 1804],
        [5879,  320],
        [5889, 3180],
        [ 668, 2565],
        [2123, 1042],
        [4139, 2546],
        [5658, 1417],
        [1916, 1670],
        [4907,  522],
        [2155, 2684],
        [3285, 3618],
        [3712, 1238],
        [4509, 2273],
        [5840,   22],
        [2851, 2276],
        [3311, 3318],
        [5344, 2163],
        [3650,   39],
        [3410, 2017],
        [2977, 3043],
        [5918, 1852],
        [4025, 3166],
        [5

## GMF模型
这里建立GMF模型， 这个模型的输入就是用户和物品的ID， 然后通过Embedding层得到它的向量， 然后就可以加权(过一个全连接层)得到最后的输出。<br>

![](img/3.png)

In [10]:
class GMF(nn.Module):
    
    def __init__(self, num_users, num_items, latent_dim, regs=[0, 0]):
        super(GMF, self).__init__()
        self.MF_Embedding_User = nn.Embedding(num_embeddings=num_users, embedding_dim=latent_dim)
        self.MF_Embedding_Item = nn.Embedding(num_embeddings=num_items, embedding_dim=latent_dim)
        self.linear = nn.Linear(latent_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, inputs):
        # 这个inputs是一个批次的数据， 所以后面的操作切记写成inputs[0], [1]这种， 这是针对某个样本了， 我们都是对列进行的操作
        
        # 先把输入转成long类型
        inputs = inputs.long()
        
        # 用户和物品的embedding
        MF_Embedding_User = self.MF_Embedding_User(inputs[:, 0])  # 这里踩了个坑， 千万不要写成[0]， 我们这里是第一列
        MF_Embedding_Item = self.MF_Embedding_Item(inputs[:, 1])
        
        # 两个隐向量点积
        predict_vec = torch.mul(MF_Embedding_User, MF_Embedding_Item)
        
        # liner
        linear = self.linear(predict_vec)
        output = self.sigmoid(linear)
        output =output.squeeze(-1)
        return output

In [11]:
# 看一下这个网络
model = GMF(1, 1, 10)
summary(model, input_shape=(2,))

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Embedding-1                                 [-1, 10]                   10
Embedding-2                                 [-1, 10]                   10
Linear-3                                     [-1, 1]                   11
Sigmoid-4                                    [-1, 1]                    0
Total params: 31
Trainable params: 31
Non-trainable params: 0
--------------------------------------------------------------------------
Input size (MB): 0.000008
Forward/backward pass size (MB): 0.000168
Params size (MB): 0.000118
Estimated Total Size (MB): 0.000294
--------------------------------------------------------------------------


'--------------------------------------------------------------------------\nLayer (type)                            Output Shape              Param #\n==========================================================================\nEmbedding-1                                 [-1, 10]                   10\nEmbedding-2                                 [-1, 10]                   10\nLinear-3                                     [-1, 1]                   11\nSigmoid-4                                    [-1, 1]                    0\n==========================================================================\nTotal params: 31\nTrainable params: 31\nNon-trainable params: 0\n--------------------------------------------------------------------------\nInput size (MB): 0.000008\nForward/backward pass size (MB): 0.000168\nParams size (MB): 0.000118\nEstimated Total Size (MB): 0.000294\n--------------------------------------------------------------------------'

## 建立模型 

In [12]:
## 设置
model = GMF(num_users, num_items, num_factors)
model.to(device)

GMF(
  (MF_Embedding_User): Embedding(6040, 8)
  (MF_Embedding_Item): Embedding(3706, 8)
  (linear): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [13]:
num_items

3706

In [14]:
# 简单测试一下模型
for (x, y) in iter(dl_train):
    #x = x.cuda()
    print(model(x))
    break

tensor([0.3138, 0.2002, 0.4077, 0.4507, 0.2413, 0.3409, 0.7928, 0.4120, 0.4815,
        0.3275, 0.4801, 0.2426, 0.4884, 0.5308, 0.5502, 0.4265, 0.2749, 0.1516,
        0.3524, 0.4576, 0.2948, 0.2027, 0.3754, 0.2628, 0.6651, 0.5426, 0.3618,
        0.3468, 0.4321, 0.4631, 0.3461, 0.3368, 0.3381, 0.4172, 0.4149, 0.2961,
        0.6116, 0.5491, 0.4271, 0.2765, 0.5374, 0.3583, 0.4881, 0.3211, 0.3423,
        0.5648, 0.5226, 0.5944, 0.3890, 0.2747, 0.5891, 0.1737, 0.4172, 0.6443,
        0.4481, 0.3728, 0.4477, 0.2576, 0.2282, 0.3318, 0.4871, 0.4477, 0.5589,
        0.1703], grad_fn=<SqueezeBackward1>)


## 模型的训练与评估

### 模型评估函数

In [15]:
# Global variables that are shared across processes
_model = None
_testRatings = None
_testNegatives = None
_K = None

# HitRation
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

# NDCG
def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return np.log(2) / np.log(i+2)
    return 0

def eval_one_rating(idx):   # 一次评分预测
    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    
    # Get prediction scores
    map_item_score = {}
    users = np.full(len(items), u, dtype='int32')
    
    test_data = torch.tensor(np.vstack([users, np.array(items)]).T).to(device)
    predictions = _model(test_data)
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i].data.cpu().numpy()
    items.pop()
    
    # Evaluate top rank list
    ranklist = heapq.nlargest(_K, map_item_score, key=lambda k: map_item_score[k])  # heapq是堆排序算法， 取前K个
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return hr, ndcg

def evaluate_model(model, testRatings, testNegatives, K):
    """
    Evaluate the performance (Hit_Ratio, NDCG) of top-K recommendation
    Return: score of each test rating.
    """
    global _model
    global _testRatings
    global _testNegatives
    global _K
    
    _model = model
    _testNegatives = testNegatives
    _testRatings = testRatings
    _K = K
    
    hits, ndcgs = [], []
    for idx in range(len(_testRatings)):
        (hr, ndcg) = eval_one_rating(idx)
        hits.append(hr)
        ndcgs.append(ndcg)
    return hits, ndcgs   

### 模型的训练

In [16]:
# 训练参数设置
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

In [17]:
# 计算出初始的评估
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK)

In [18]:
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print('Init: HR=%.4f, NDCG=%.4f' %(hr, ndcg))

Init: HR=0.1086, NDCG=0.0500


In [19]:
# 模型训练 
best_hr, best_ndcg, best_iter = hr, ndcg, -1

epochs = 20
log_step_freq = 10000

for epoch in range(epochs):
    
    # 训练阶段
    model.train()
    loss_sum = 0.0
    for step, (features, labels) in enumerate(dl_train, 1):
        
        #features, labels = features.cuda(), labels.cuda()
        # 梯度清零
        optimizer.zero_grad()
        
        # 正向传播
        predictions = model(features)
        loss = loss_func(predictions, labels)
        
        # 反向传播求梯度
        loss.backward()
        optimizer.step()
        
        # 打印batch级别日志
        loss_sum += loss.item()
        if step % log_step_freq == 0:
            print(("[step = %d] loss: %.3f") %
                  (step, loss_sum/step))
    
    # 验证阶段
    model.eval()
    (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK)
    hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    if hr > best_hr:
        best_hr, best_ndcg, best_iter = hr, ndcg, epoch
        torch.save(model.state_dict(), 'Pre_train/m1-1m_GMF.pkl')  
        
    info = (epoch, loss_sum/step, hr, ndcg)
    print(("\nEPOCH = %d, loss = %.3f, hr = %.3f, ndcg = %.3f") %info)
print('Finished Training...') 

[step = 10000] loss: 0.507
[step = 20000] loss: 0.504
[step = 30000] loss: 0.503
[step = 40000] loss: 0.502
[step = 50000] loss: 0.502
[step = 60000] loss: 0.502
[step = 70000] loss: 0.501

EPOCH = 0, loss = 0.501, hr = 0.111, ndcg = 0.051
[step = 10000] loss: 0.499
[step = 20000] loss: 0.500
[step = 30000] loss: 0.498
[step = 40000] loss: 0.490
[step = 50000] loss: 0.476
[step = 60000] loss: 0.462
[step = 70000] loss: 0.449

EPOCH = 1, loss = 0.441, hr = 0.414, ndcg = 0.226
[step = 10000] loss: 0.360
[step = 20000] loss: 0.359
[step = 30000] loss: 0.359
[step = 40000] loss: 0.358
[step = 50000] loss: 0.357
[step = 60000] loss: 0.357
[step = 70000] loss: 0.356

EPOCH = 2, loss = 0.356, hr = 0.451, ndcg = 0.247
[step = 10000] loss: 0.345
[step = 20000] loss: 0.344
[step = 30000] loss: 0.343
[step = 40000] loss: 0.342
[step = 50000] loss: 0.341
[step = 60000] loss: 0.340
[step = 70000] loss: 0.339

EPOCH = 3, loss = 0.337, hr = 0.483, ndcg = 0.267
[step = 10000] loss: 0.323
[step = 20000